In [1]:
import json
import joblib
from pathlib import Path
from timeit import default_timer as timer

import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model
from torch.optim.lr_scheduler import OneCycleLR

from accelerate import Accelerator

from isic_helper import DotDict
from isic_helper import get_folds
from isic_helper import compute_auc, compute_pauc
from isic_helper import set_seed
from isic_helper import time_to_str

In [2]:
cfg = DotDict()
cfg.infer = False
cfg.cpu = False
cfg.mixed_precision = None

cfg.image_size = 384
cfg.lr = 5e-4
cfg.num_epochs = 2
cfg.seed = 2022
cfg.train_batch_size = 32
cfg.train_num_worker = 2
cfg.val_batch_size = 256
cfg.val_num_worker = 2
# cfg.log_every = 10

cfg.models_output_dir = "models"
cfg.model_name = "resnet18_v1"

In [3]:
INPUT_PATH = Path("../input/isic-2024-challenge/")
MODELS_OUTPUT_PATH = Path(f"{cfg.models_output_dir}")
MODELS_OUTPUT_PATH.mkdir(exist_ok=True)

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
train_images = h5py.File(INPUT_PATH / "train-image.hdf5", mode="r")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
train_metadata = train_metadata.sample(frac=0.05, random_state=cfg.seed)
print(f"Train data size: {train_metadata.shape}")

Train data size: (20053, 57)


In [4]:
id_column = "isic_id"
target_column = "final_target"
group_column = "patient_id"

train_ids = train_metadata[id_column]
groups = train_metadata[group_column]
folds = train_metadata["fold"]
y_train = train_metadata[target_column]

In [5]:
accelerator = Accelerator(cpu=cfg.cpu, mixed_precision=cfg.mixed_precision, device_placement=True)

In [6]:
def dev_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(
            mean=[0., 0., 0.],
            std=[1, 1, 1],
            max_pixel_value=255.0,
            p=1.0
        ),
        ToTensorV2()
    ], p=1.)
    return transform

def val_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(
            mean=[0., 0., 0.],
            std=[1, 1, 1],
            max_pixel_value=255.0,
            p=1.0
        ),
        ToTensorV2()
    ], p=1.)
    return transform

class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        data = self.metadata.iloc[index]
        
        image = np.array(Image.open(BytesIO(self.images[data[id_column]][()])))
        image = self.augment(image=image)["image"]
        
        record = {
            "index": index,
            "image": image
        }
        
        if not self.infer:
            target = data[target_column]
            record["target"] = target
        
        return record

class ISICNet(nn.Module):
    def __init__(self, arch="resnet18", pretrained=False, infer=False):
        super(ISICNet, self).__init__()
        self.infer = infer
        self.model = create_model(model_name=arch, pretrained=pretrained, in_chans=3,  num_classes=0, global_pool='')
        self.classifier = nn.Linear(self.model.num_features, 1)
        
        self.dropout = nn.ModuleList([nn.Dropout(0.5) for i in range(5)])
        
    def forward(self, batch):
        image = batch["image"]
        
        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs,-1)
        
        if self.training:
            logit=0
            for i in range(len(self.dropout)):
                logit += self.classifier(self.dropout[i](pool))
            logit = logit/len(self.dropout)
        else:
            logit = self.classifier(pool)
            
        output = {}
        output["preds"] = torch.sigmoid(logit)
        
        if not self.infer:
            target = batch["target"].unsqueeze(1)
            output["bce_loss"] = F.binary_cross_entropy_with_logits(logit.float(), target.float())
            
        return output

In [7]:
best_num_epochs = {}
val_auc_scores = {}
val_pauc_scores = {}
all_folds = np.sort(folds.unique())
oof_predictions = np.zeros(train_metadata.shape[0])
for fold in all_folds:
    set_seed(cfg.seed)
    
    print(f"Running fold: {fold}")
    dev_index = folds != fold
    val_index = folds == fold
    
    dev_metadata = train_metadata[train_metadata["fold"] != fold].reset_index(drop=True);print(f"Dev data size: {dev_metadata.shape}")
    val_metadata = train_metadata[train_metadata["fold"] == fold].reset_index(drop=True);print(f"Val data size: {val_metadata.shape}")
    
    dev_dataset = ISICDataset(dev_metadata, train_images, augment=dev_augment(image_size=cfg.image_size))
    val_dataset = ISICDataset(val_metadata, train_images, augment=val_augment(image_size=cfg.image_size))

    dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=cfg.train_batch_size, num_workers=cfg.train_num_worker)
    val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=cfg.val_batch_size, num_workers=cfg.val_num_worker, drop_last=False)
    
    net = ISICNet(pretrained=True)
    net = net.to(accelerator.device)
    
    for param in net.parameters():
        param.requires_grad = False

    for param in net.classifier.parameters():
        param.requires_grad = True
        
    optimizer = torch.optim.Adam(params=net.model.parameters(), lr=cfg.lr / 25)
    lr_scheduler = OneCycleLR(optimizer=optimizer, max_lr=cfg.lr, epochs=cfg.num_epochs, steps_per_epoch=len(dev_dataloader))

    net, optimizer, dev_dataloader, val_dataloader, lr_scheduler = accelerator.prepare(
        net, optimizer, dev_dataloader, val_dataloader, lr_scheduler
    )
    
    print("Ready to train")
    
    overall_step = 0
    starting_epoch = 0
    best_pauc_score = -np.Inf
    best_auc_score = -np.Inf
    best_epoch = None
    best_val_preds = None

    for epoch in range(starting_epoch, cfg.num_epochs):
        net.train()
        count = 0
        start_timer = timer()
        for step, batch in enumerate(dev_dataloader):
            output = net(batch)
            loss = output["bce_loss"]
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            overall_step += 1
            count += len(batch["index"])
#             if step % cfg.log_every == 0:
            print(f'\rTraining: {count}/{len(dev_dataset)}',
                      time_to_str(timer() - start_timer, "min"),
                      end='', flush=True) 

        net.eval()
        val_preds = []
        val_y = []
        count = 0
        start_timer = timer()
        for step, batch in enumerate(val_dataloader):
            with torch.no_grad():
                output = net(batch)
            val_preds_batch = output["preds"]
            val_preds.append(val_preds_batch.data.cpu().numpy().reshape(-1))
            val_y_batch = batch["target"]
            val_y.append(val_y_batch.data.cpu().numpy().reshape(-1))
            count += len(batch["index"])
#             if step % cfg.log_every == 0:
            print(f'\rValidation: {count}/{len(val_dataset)}',
                      time_to_str(timer() - start_timer, "min"),
                      end='', flush=True)

        print('')
        val_preds = np.concatenate(val_preds)
        val_y = np.concatenate(val_y)
        auc = compute_auc(val_y, val_preds) 
        pauc = compute_pauc(val_y, val_preds, min_tpr=0.80)
        
        if pauc >= best_pauc_score:
            best_auc_score = auc
            best_pauc_score = pauc
            best_epoch = epoch
            best_val_preds = val_preds
        print(f"Epoch pauc: {pauc} | Best auc: {best_auc_score} | Best pauc: {best_pauc_score} | Best epoch: {best_epoch}")
        
        output_dir = f"fold_{fold}/model_{cfg.model_name}_epoch_{epoch}"
        if cfg.models_output_dir is not None:
            output_dir = Path(f"{cfg.models_output_dir}/{output_dir}")
        accelerator.save_state(output_dir)
    
    best_num_epochs[f"fold_{fold}"] = best_epoch
    val_auc_scores[f"fold_{fold}"] = best_auc_score
    val_pauc_scores[f"fold_{fold}"] = best_pauc_score
    
    oof_predictions[val_index] = best_val_preds
    print("\n")

Running fold: 1
Dev data size: (16040, 57)
Val data size: (4013, 57)


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Ready to train
Validation: 4013/4013 0 min 20.92 sec
Epoch pauc: 0.17268195413758722 | Best auc: 0.8634097706879361 | Best pauc: 0.17268195413758722 | Best epoch: 0
Validation: 4013/4013 0 min 16.25 sec
Epoch pauc: 0.1628115653040877 | Best auc: 0.8634097706879361 | Best pauc: 0.17268195413758722 | Best epoch: 0


Running fold: 2
Dev data size: (16037, 57)
Val data size: (4016, 57)
Ready to train
Validation: 4016/4016 0 min 17.09 sec
Epoch pauc: 0.046399202591577364 | Best auc: 0.6194036049505773 | Best pauc: 0.046399202591577364 | Best epoch: 0
Validation: 4016/4016 0 min 16.07 sec
Epoch pauc: 0.07430849738350359 | Best auc: 0.6311986045352603 | Best pauc: 0.07430849738350359 | Best epoch: 1


Running fold: 3
Dev data size: (16105, 57)
Val data size: (3948, 57)
Ready to train
Validation: 3948/3948 0 min 16.24 sec
Epoch pauc: 0.012924480486568673 | Best auc: 0.2461986822098327 | Best pauc: 0.012924480486568673 | Best epoch: 0
Validation: 3948/3948 0 min 15.67 sec
Epoch pauc: 0.01368474

In [8]:
oof_preds_df = pd.DataFrame({
    id_column: train_ids,
    group_column: groups,
    "fold": folds,
    target_column: y_train,
    f"oof_{cfg.model_name}": oof_predictions
})
oof_preds_df.to_csv(f"oof_preds_{cfg.model_name}.csv")
oof_preds_df.head()

,isic_id,patient_id,fold,final_target,oof_resnet18_v1
139465,ISIC_3531602,IP_9742855,2,0,0.524050
155858,ISIC_3939264,IP_0028775,3,0,0.556670
299181,ISIC_7485201,IP_3864041,4,0,0.502346
344904,ISIC_8613136,IP_3549978,3,0,0.520417
176332,ISIC_4447313,IP_4549819,4,0,0.535450


In [9]:
best_num_epochs

{'fold_1': 0, 'fold_2': 1, 'fold_3': 1, 'fold_4': 0, 'fold_5': 0}

In [10]:
val_auc_scores

{'fold_1': 0.8634097706879361,
 'fold_2': 0.6311986045352603,
 'fold_3': 0.23859604662949824,
 'fold_4': 0.6758323057953144,
 'fold_5': 0.3858993522670653}

In [11]:
val_pauc_scores

{'fold_1': 0.17268195413758722,
 'fold_2': 0.07430849738350359,
 'fold_3': 0.013684744044602126,
 'fold_4': 0.07916152897657212,
 'fold_5': 0.0075236671649227685}

In [12]:
cv_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"])
cv_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"], min_tpr=0.8)

cv_auc_avg = np.mean(list(val_auc_scores.values()))
cv_pauc_avg = np.mean(list(val_pauc_scores.values()))

In [13]:
print(f"CV AUC OOF: {cv_auc_oof}")
print(f"CV PAUC OOF: {cv_pauc_oof}")
print(f"CV AUC AVG: {cv_auc_avg}")
print(f"CV PAUC AVG: {cv_pauc_avg}")

CV AUC OOF: 0.4961077844311378
CV PAUC OOF: 0.01958928297251649
CV AUC AVG: 0.5589872159830149
CV PAUC AVG: 0.06947207834143757


In [14]:
params = vars(cfg)
params = {k: v for k, v in params.items() if not k.startswith("_")}

metadata = {
    "params": params,
    "best_num_epochs": best_num_epochs,
    "val_auc_scores": val_auc_scores,
    "val_pauc_scores": val_pauc_scores,
    "cv_auc_oof": cv_auc_oof,
    "cv_pauc_oof": cv_pauc_oof,
    "cv_auc_avg": cv_auc_avg,
    "cv_pauc_avg": cv_pauc_avg
}

with open("run_metadata.json", "w") as f:
    json.dump(metadata, f)